In [1]:
import torch
import yaml

# Local imports
import carl_utils.preprocessing as carl_ppro
import carl_utils.train as carl_train

import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.version.cuda)
#DEVICE = 'cpu'
print(DEVICE)

11.7
cuda:0


In [2]:
total_features = yaml.load(open("carl_features.yaml", 'r'), Loader=yaml.CLoader)
total_features

{'Electron_set': {'set': True,
  'size': 4,
  'subfeatures': ['Electron_pt',
   'Electron_eta',
   'Electron_phi',
   'Electron_mass']},
 'FatJet_set': {'set': True,
  'size': 5,
  'subfeatures': ['FatJet_pt',
   'FatJet_eta',
   'FatJet_phi',
   'FatJet_mass',
   'FatJet_btagCSVV2']},
 'Jet_set': {'set': True,
  'size': 5,
  'subfeatures': ['Jet_pt',
   'Jet_eta',
   'Jet_phi',
   'Jet_mass',
   'Jet_btagCSVV2']},
 'MET_sumEt': {'set': False, 'size': 1, 'subfeatures': ['MET_sumEt']},
 'Muon_set': {'set': True,
  'size': 4,
  'subfeatures': ['Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_mass']},
 'Photon_set': {'set': True,
  'size': 4,
  'subfeatures': ['Photon_pt', 'Photon_eta', 'Photon_phi', 'Photon_mass']}}

In [3]:
subfeatures = ["Jet_set", "Electron_set"]#, "Muon_set"]
features = dict(zip(subfeatures, [total_features[feat] for feat in subfeatures]))
features

{'Jet_set': {'set': True,
  'size': 5,
  'subfeatures': ['Jet_pt',
   'Jet_eta',
   'Jet_phi',
   'Jet_mass',
   'Jet_btagCSVV2']},
 'Electron_set': {'set': True,
  'size': 4,
  'subfeatures': ['Electron_pt',
   'Electron_eta',
   'Electron_phi',
   'Electron_mass']}}

In [4]:
weight_features = ["genWeight", "btagWeight_CSVV2"]

## Initiate the datasets and load the data

In [11]:
n_train_events = int(1e6) // 2
n_val_events = int(2e5) // 2

train_nominal_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_nominal_DeepSets_training_data_new2.root",], features, 0, stop_event=n_train_events)
valid_nominal_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_nominal_DeepSets_validation_data_new2.root",], features, 0, stop_event=n_val_events)

train_PS_var_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_PS_var_DeepSets_training_data_new2.root",], features, 1, stop_event=n_train_events)
valid_PS_var_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_PS_var_DeepSets_validation_data_new2.root",], features, 1, stop_event=n_val_events)

100%|██████████| 100000/100000 [00:35<00:00, 2853.66it/s]


## Prepare the data

In [12]:
train_generator_data = carl_ppro.CombinedDataset(train_nominal_dataset, train_PS_var_dataset)
valid_generator_data = carl_ppro.CombinedDataset(valid_nominal_dataset, valid_PS_var_dataset)

## Define the settings of the model and training

In [13]:
hidden1 = 512 // 2
hidden2 = 256 // 2
hidden3 = 128 // 2
classify1 = 1024  // 2
classify2 = 1024 // 2
outputs = 1

model_settings = {
    "features": features,
    "phi": [hidden1, hidden2, hidden3],
    "mlp": [classify1, classify2]
}


training_settings = {
    "optimizer": "Adam",
    "learning_rate": 1e-2,
    "batch_size": 256,
    "n_epochs": 30,
    "patience": 6,
    "device": DEVICE,
    "n_events": n_train_events * 2,
    "saveAs": "models/deepsets_agc_model_new6"
}

# Perform the training

In [ ]:
model, losses = carl_train.train(model_settings, train_generator_data, valid_generator_data, **training_settings)

Constructing the model
Loading the input data scaling


100%|██████████| 977/977 [00:11<00:00, 85.79it/s] 


Training the model


Epoch: 01, Training Loss:   0.6862
           Validation Loss: 0.6828


  3%|▎         | 1/30 [04:23<2:07:10, 263.12s/it]

...

 57%|█████▋    | 17/30 [1:15:10<57:08, 263.75s/it]  

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

New best model saved to: models/deepsets_agc_model_new6.zip


 60%|██████    | 18/30 [1:19:24<52:12, 261.05s/it]                 

Epoch: 18, Training Loss:   0.6795
           Validation Loss: 0.6809
Stale epoch


loss = 0.68566:  44%|████▍     | 1721/3907 [01:48<02:11, 16.63it/s]

In [ ]:
model

In [ ]:
losses

# Check the ONNX Model

In [2]:
import carl_utils.models as carl_models
import numpy as np
import onnx, onnxruntime
onnxruntime.get_device()

'GPU'

In [3]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [4]:
#DEVICE = 'cpu'
model_name = "models/deepsets_agc_model_new6"

model = carl_models.load_model("{}.zip".format(model_name)).to(DEVICE)

onnx_model = carl_models.load_onnx_model("{}.zip".format(model_name))
onnx.checker.check_model(onnx_model)
ort_session = carl_models.load_onnx_session("{}.zip".format(model_name))

2023-09-07 15:38:29.401368892 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:614 CreateExecutionProviderInstance] Failed to create TensorrtExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/TensorRT-ExecutionProvider.html#requirements to ensure all dependencies are met.


In [5]:
test_input = []
input_names = []
for k in model.features:
    test_input.append(torch.randn(1, model.features[k]["size"], requires_grad=True).T[None, :].to(DEVICE))
    input_names.append(k)
test_input.append(torch.ones(len(model.features), 1, dtype=int).to(DEVICE))
input_names.append("sample_indices")
test_input = tuple(test_input)

In [6]:
for i in range(1000):
    test_input = []
    input_names = []
    for k in model.features:
        test_input.append(torch.randn(1, model.features[k]["size"], requires_grad=True).T[None, :].to(DEVICE))
        input_names.append(k)
    test_input.append(torch.ones(len(model.features), 1, dtype=int).to(DEVICE))
    input_names.append("sample_indices")
    test_input = tuple(test_input)

    # Compute PyTorch output prediction
    with torch.no_grad():
        model.eval()
        torch_out = model(*test_input)
    
    # compute ONNX Runtime output prediction
    ort_inputs = {x.name: to_numpy(test_input[i]) for i, x in enumerate(ort_session.get_inputs())}
    ort_outs = ort_session.run(None, ort_inputs)
    
    # compare ONNX Runtime and PyTorch results
    np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

In [11]:
%timeit model(*test_input)

872 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
%timeit ort_session.run(None, ort_inputs)

361 µs ± 6.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
